# Ryan image

In [ ]:
from scipy import stats
import scipy.ndimage as ndimage
import astropy
import math
import time

#libraries for image registration
import FITS_tools
from FITS_tools.hcongrid import hcongrid

#import relevant spline libraries
from scipy.interpolate import Rbf

#####UPDATE INFORMATION HERE####
#DO YOU WANT TO FLAT FIELD AND BIAS SUBTRACT?
biassub = 0 # yes = 1 no = 0 to bias subtract
flatdiv = 0 # yes = 1 no = 0 to flat field
align = 1# yes = 1 no = 0 to align based on coordinates

#useful directories
rawdir = '/home/'+user+'/Documents/Mestrado/FFIs/8_golden/teste/' #directory with the raw images
cdedir = '/home/'+user+'/dia_kepler/mychanges/Python/' #directory where the code 'lives'
clndir = '/home/'+user+'/Documents/Mestrado/FFIs/8_golden/teste/teste_clean/'

#sample every how many pixels? usually 32x32 is OK but it can be larger or smaller
pix1 = 64 # UPDATE HERE FOR BACKGROUND SPACING
pix2 = 55
axs1 = 1024 # NAXIS 2 = number of lines
axs2 = 1100 # NAXIS 1 = number of columns

#get the image list and the number of files which need reduction
os.chdir(rawdir) #changes to the raw image direcotory
files = [f for f in glob.glob("*.fits") if isfile(join(rawdir, f))] #gets the relevant files with the proper extension
files.sort()
nfiles = len(files)
os.chdir(cdedir) #changes back to the code directory
hdulist = fits.open(rawdir+files[0])
n_images = len(hdulist)-1

ref, rhead = fits.getdata(rawdir+files[0], header = True)

rhead['NAXIS1'] = 1100
rhead['NAXIS2'] = 1024

#sample every how many pixels?
bxs1 = 256 #how big do you want to make the BIG boxes for each image?
bxs2 = 275
lop = pix1+pix2
sze = (bxs1/pix1)*(bxs2/pix2)+(bxs1/pix1)+(bxs2/pix2)+1 #size holder for later

#begin cleaning
hld = files[0].split('.') #gives an array like ['kplr2009114174833_ffi-cal', 'fits'
# #update the name to be appropriate for what was done to the file
finnme= "ryan.fits"  

header = hdulist[1].header
w=WCS(header)
orgimg = hdulist[1].data

# orgimg, header = fits.getdata(rawdir+files[ii], header = True)

res = np.zeros(shape=(axs2, axs1)) #holder for the background 'image'
bck = np.zeros(shape=int((axs1/bxs1)*(axs2/bxs2))) #get the holder for the image background
sbk = np.zeros(shape=int((axs1/bxs1)*(axs2/bxs2))) #get the holder for the sigma of the image background

tts = 0
np.shape(cut)

In [ ]:
for oo in range(0, axs2, bxs2):#axs1=1100 axs2=1024
    for ee in range(0, axs1, bxs1):
        if tts<len(bck):
            img = bigimg[ee:ee+bxs1,oo:oo+bxs2] #split the image into small subsections
            
            #calculate the sky statistics
            cimg, clow, chigh = scipy.stats.sigmaclip(img, low=2.5, high = 2.5) #do a 2.5 sigma clipping
            sky = numpy.median(cimg) #determine the sky value
            sig = numpy.std(cimg) #determine the sigma(sky)

            bck[tts] = sky #insert the image median background
            sbk[tts] = sig #insert the image sigma background

            #create holder arrays for good and bad pixels
            x = np.zeros(shape=int(sze))
            y = np.zeros(shape=int(sze))
            v = np.zeros(shape=int(sze))
            s = np.zeros(shape=int(sze))
            nd = 0

            #begin the sampling of the "local" sky value
            for jj in range(0, bxs2 + pix2, pix2):
                for kk in range(0,bxs1 + pix1, pix1):
                    if nd<len(x):
                        #il = np.amax([kk-lop,0])
                        il = np.amax([kk-2*pix1,0])
                        #ih = np.amin([kk+lop, bxs1-1])
                        ih = np.amin([kk+2*pix1, bxs1-1])
                        #jl = np.amax([jj-lop, 0])
                        jl = np.amax([jj-2*pix2, 0])
                        #jh = np.amin([jj+lop, bxs2-1])
                        jh = np.amin([jj+2*pix2, bxs2-1])
                        c = img[il:ih, jl:jh]
                        #print(il,ih,jl,jh)

                        #print(np.shape(c))

                        #select the median value with clipping
                        mmean, lsky, ssig = sigma_clipped_stats(c, sigma=2.5)
                        #print (jj)
                        #print(np.amin([jj, bxs2]))
                        x[nd] = np.amin([kk, bxs1-1]) #determine the pixel to input
                        y[nd] = np.amin([jj, bxs2-1]) #determine the pixel to input
                        v[nd] = lsky #median sky
                        s[nd] = ssig #sigma sky
                        nd = nd + 1

            #now we want to remove any possible values which have bad sky values
            rj = np.where(v <= 0) #stuff to remove
            kp = np.where(v > 0) #stuff to keep

            if (len(rj[0]) > 0):
                #keep only the good points
                xgood = x[kp]
                ygood = y[kp]
                vgood = v[kp]
                sgood = s[kp]
                print(ygood)
                for jj in range(0, len(rj[0])):
                    #select the bad point
                    xbad = x[rj[jj]]
                    ybad = y[rj[jj]]
                    print(xbad,ybad)
                    #use the distance formula to get the closest points
                    rd = math.sqrt((xgood-ygood)**2.+(ygood-ybad)**2.)
                    #sort the radii
                    pp = sorted(range(len(rd)), key = lambda k:rd[k])
                    #use the closest 10 points to get a median
                    vnear = vgood[pp[0:9]]
                    ave = np.median(vnear)
                    #insert the good value into the array
                    v[rj[jj]] = ave

            #now we want to remove any possible values which have bad sigmas
            rjs = np.where(s >= 2*sig)
            rj  = rjs[0]
            kps = np.where(s < 2*sig)
            kp  = kps[0]
            
            if (len(rj) > 0):
                #keep only the good points
                xgood = np.array(x[kp])
                ygood = np.array(y[kp])
                vgood = np.array(v[kp])
                sgood = np.array(s[kp])

                for jj in range(0, len(rj)):
                    #select the bad point
                    xbad = x[rj[jj]]
                    ybad = y[rj[jj]]
                    #print xbad, ybad
                    #use the distance formula to get the closest points
                    rd = np.sqrt((xgood-xbad)**2.+(ygood-ybad)**2.)
                    #sort the radii
                    pp = sorted(range(len(rd)), key = lambda k:rd[k])
                    #use the closest 10 points to get a median
                    vnear = vgood[pp[0:9]]
                    ave = np.median(vnear)
                    #insert the good value into the array
                    v[rj[jj]] = ave
                        
            #now we interpolate to the rest of the image with a thin-plate spline    
            xi = np.linspace(0, bxs1-1, bxs1)
            yi = np.linspace(0, bxs2-1, bxs2)
            XI, YI = np.meshgrid(xi, yi)
            XI = XI.T
            YI = YI.T
            rbf = Rbf(x, y, v, function = 'thin-plate', smooth = 0.0)
            reshld = rbf(XI, YI)
            print(np.shape(XI),np.shape(YI))
            print(np.shape(reshld))
'''
             #now add the values to the residual image
            res[ee:ee+bxs1, oo:oo+bxs2] = reshld
            tts = tts+1

        #get the median background
        mbck = np.median(bck)
        sbck = np.median(sbk)

        #subtract the sky gradient and add back the median background
        sub = bigimg-res
        sub = sub + mbck

         #align the image
        algn = hcongrid(sub, header, rhead)
        #update the header
        header['CTYPE1'] = rhead['CTYPE1']
        header['CTYPE2'] = rhead['CTYPE2']
        header['CRVAL1'] = rhead['CRVAL1']
        header['CRVAL2'] = rhead['CRVAL2']
        header['CRPIX1'] = rhead['CRPIX1']
        header['CRPIX2'] = rhead['CRPIX2']
        header['CD1_1'] = rhead['CD1_1']
        header['CD1_2'] = rhead['CD1_2']
        header['CD2_1'] = rhead['CD2_1']
        header['CD2_2'] = rhead['CD2_2']
        #update the header
        header['medback'] = mbck
        header['sigback'] = sbck
        header['bksub'] = 'yes'
        if (align == 1):
            header['align'] = 'yes'
        #write out the subtraction
        shd = fits.PrimaryHDU(algn, header=header)
        shd.writeto(clndir+finnme, overwrite = True)
        #stop the watch
        fn = time.time()
        print ('Background subtraction for '+files[ii]+' finished in '+str(fn-st)+'s.')

# print ('All done! See ya later alligator.')
#'''

In [1]:
# user = input('Insert the name of your user')
# print('the user is',user)